# LeverageJustAFewKeywords

https://github.com/aqweteddy/LeverageJustAFewKeywords

This notebook is for making things and code in **unofficial repository** more clear, looking inside the data structure.

**Experiments are run using Colab**

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
!pip install transformers
!pip install mittens

     |████████████████████████████████| 2.5MB 20.8MB/s 
     |████████████████████████████████| 901kB 38.1MB/s 
     |████████████████████████████████| 3.3MB 39.6MB/s 


In [ ]:
%cd /content/drive/MyDrive/LeverageJustAFewKeywords

/content/drive/MyDrive/LeverageJustAFewKeywords


In [ ]:
%cd ../LeverageJustAFewKeywords

/content/drive/MyDrive/LeverageJustAFewKeywords


In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np

# SummaryWriter

In [1]:
from tensorboardX import SummaryWriter
from utils import *

In [2]:
log_dir = './experiments'
writer = SummaryWriter(log_dir)

In [3]:
a = [5, 8, 7]
a_dict = list2dict(a)
a_dict

{'0': 5, '1': 8, '2': 7}

In [4]:
writer.add_scalars('test', a_dict, 0)

# word2vec

In [ ]:
from train import Trainer
from config import hparams

In [ ]:
trainer = Trainer(hparams, 'cuda')

INFO:root:loading dataset...
INFO:gensim.models.utils_any2vec:loading projection weights from ../wv/w2v_corpus_trained.bin
INFO:gensim.models.utils_any2vec:loaded (19379, 300) matrix from ../wv/w2v_corpus_trained.bin
INFO:gensim.models.utils_any2vec:loading projection weights from ../wv/w2v_corpus_trained.bin
asp_category: 9
bag_dim: 226
loading dataset...
INFO:gensim.models.utils_any2vec:loaded (19379, 300) matrix from ../wv/w2v_corpus_trained.bin
INFO:root:dataset_size: 588229
INFO:root:loading model...
asp_category: 9
bag_dim: 226
length of data: 653; 	length_of_labels: 653
INFO:gensim.models.utils_any2vec:loading projection weights from ../wv/w2v_corpus_trained.bin
INFO:gensim.models.utils_any2vec:loaded (19379, 300) matrix from ../wv/w2v_corpus_trained.bin


In [ ]:
print(trainer.student.emb.weight.shape)
print(trainer.student.emb.weight[2])

torch.Size([19381, 300])
tensor([-0.0046, -0.0121, -0.2932,  0.3288, -0.8590, -1.4005,  0.6807, -0.6227,
         0.5384, -0.2821,  0.2095,  1.3155, -0.5457, -0.0148,  0.2519, -0.3193,
         1.0681, -0.8513, -0.8479,  1.8756, -1.1531, -0.7611,  0.0174,  0.7811,
        -0.1596,  0.5381,  0.3012,  0.1297, -0.2413,  0.0116, -0.0300, -0.0506,
        -0.1058, -0.3877,  1.7364, -0.0345,  0.3005, -0.5975,  0.3802, -1.0362,
        -1.0269,  1.7519,  0.3192, -0.8838,  0.5075,  0.6185,  0.5142,  0.2189,
        -0.9033,  0.1991, -0.4957,  1.3350, -0.9134,  0.4006,  0.1834,  1.0811,
        -1.0150, -0.2774, -1.1022,  0.6572,  0.5651,  0.3056,  0.9317, -0.4576,
        -0.0701,  0.1628, -0.0856, -0.1371, -0.1364,  0.8349, -1.6454,  1.3656,
         0.6754,  0.8125,  0.4574, -1.1769, -0.2916,  0.5010,  0.6634,  1.4547,
         0.2387,  0.1275, -0.9204, -0.1685,  0.4335, -0.3054, -0.9092, -2.3319,
        -0.4496, -1.7536, -0.0497,  0.0915,  1.1156, -0.2939, -0.1091,  0.8205,
         0.1606

# `train.py`


## Teacher Testing

In [ ]:
# from dataset import TestDataset
from train import Trainer
import torch
from sklearn.metrics import f1_score

In [ ]:
def test(loader, model):
    score = []
    result = []
    ground = []
    for batch in loader:
        idx, bow, labels = batch
        bow = bow.cuda()
        labels = labels.cuda()
        model.z = model.reset_z()
        with torch.no_grad():
            logits = model.teacher(bow, model.z)
        result.append(logits.max(-1)[1].cpu())
        ground.append(labels.max(-1)[1].cpu())
    result = torch.cat(result, dim=0)
    ground = torch.cat(ground, dim=0)
    score = f1_score(ground.numpy(), result.numpy(), average='micro')
    # print(model.z)
    return score

In [ ]:
general_aspect = {'bags_and_cases': 4, 'bluetooth': 6, 'boots': 5, 'keyboards': 7, 'tv': 5, 'vacuums': 5}

In [ ]:
domain_range = ['bags_and_cases', 'bluetooth', 'boots', 'keyboards', 'tv', 'vacuums']
aspect_name = domain_range[5]
hparams = {
    'lr': 5e-5,
    'batch_size': 64,
    'student': {
        'pretrained': 'bert-base-uncased',
        'pretrained_dim': 768,
        'num_aspect': 9,
    },
    'description': f'{aspect_name} baseline',
    'save_dir': f'./ckpt/{aspect_name}',
    'train_file': f'./data/{aspect_name}_train.json',
    'aspect_init_file': f'./data/{aspect_name}.30.txt',
    'test_file': f'./data/{aspect_name}_test.json',
    'general_asp': general_aspect[aspect_name],
    'maxlen': 40
}

In [ ]:
trainer = Trainer(hparams, 'cuda')

INFO:root:loading dataset...


asp_category: 9
bag_dim: 239
loading dataset...


INFO:root:dataset_size: 1465525
INFO:root:loading model...


asp_category: 9
bag_dim: 239
length of data: 741; 	length_of_labels: 741


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
test(trainer.test_loader, trainer)

0.5614035087719298

## check gradient

In [ ]:
from train import *
from torch.utils.data import DataLoader

In [ ]:
hparams = {
    'lr': 5e-5,
    'batch_size': 64,
    'student': {
        'pretrained': 'bert-base-uncased',
        'pretrained_dim': 768,
        'num_aspect': 9,
    },
    'description': 'bags_and_cases baseline',
    'save_dir': './ckpt/bags',
    'aspect_init_file': './data/bags_and_cases.30.txt',
    'train_file': './data/bags_and_cases_train.json',
    'test_file': './data/bags_and_cases_test.json',
    'general_asp': 4,
    'maxlen': 40
}

In [ ]:
trainer = Trainer(hparams, 'cuda')

INFO:root:loading dataset...


asp_category: 9
bag_dim: 226
loading dataset...


INFO:root:dataset_size: 588229
INFO:root:loading model...


asp_category: 9
bag_dim: 226
653 653


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
loader = DataLoader(trainer.ds, batch_size=64)

In [ ]:
bow, id = next(iter(loader))

In [ ]:
step_loss = trainer.train_step(bow.cuda(), id.cuda())

There is no trainable parameters in teacher.

In [ ]:
for name, weight in trainer.student.named_parameters():
    if weight.requires_grad:
        if weight.grad == None:
            print(name, 'grad none')
        else:
            print(name, weight.grad.mean())
    else:
        print("don't requires grad: {}".format(name))

bert.embeddings.word_embeddings.weight tensor(-3.2293e-13, device='cuda:0')
bert.embeddings.position_embeddings.weight tensor(-3.1984e-11, device='cuda:0')
bert.embeddings.token_type_embeddings.weight tensor(-4.9671e-09, device='cuda:0')
bert.embeddings.LayerNorm.weight tensor(0.0125, device='cuda:0')
bert.embeddings.LayerNorm.bias tensor(0.0011, device='cuda:0')
bert.encoder.layer.0.attention.self.query.weight tensor(-6.6983e-05, device='cuda:0')
bert.encoder.layer.0.attention.self.query.bias tensor(0.0021, device='cuda:0')
bert.encoder.layer.0.attention.self.key.weight tensor(8.9338e-05, device='cuda:0')
bert.encoder.layer.0.attention.self.key.bias tensor(2.7759e-10, device='cuda:0')
bert.encoder.layer.0.attention.self.value.weight tensor(-7.1446e-05, device='cuda:0')
bert.encoder.layer.0.attention.self.value.bias tensor(-0.0016, device='cuda:0')
bert.encoder.layer.0.attention.output.dense.weight tensor(0., device='cuda:0')
bert.encoder.layer.0.attention.output.dense.bias tensor(-8.9

## metrics

In [ ]:
from sklearn.metrics import recall_score
from collections import Counter

In [ ]:
y_true = [0, 1, 2, 0, 1, 2]
y_pred = [0, 2, 1, 0, 0, 1]
print(Counter(y_true))
recall_score(y_true, y_pred, average='micro')


Counter({0: 2, 1: 2, 2: 2})


0.3333333333333333

In [ ]:
y_true = [0, 0, 0, 0, 0, 0]
recall_score(y_true, y_pred, average='micro')

0.5

In [ ]:
class_count = Counter(y_true)
class_count

Counter({0: 6})

## MISC

In [ ]:
0 / 1e-10

0.0

In [ ]:
import torch

In [ ]:
weights = torch.arange(3)
print(weights)
weights = torch.pow(2, weights)
print(weights)
weights = weights / (2 ** len(weights) - 1)
print(weights)
print(weights.sum())

tensor([0, 1, 2])
tensor([1, 2, 4])
tensor([0.1429, 0.2857, 0.5714])
tensor(1.)


In [ ]:
r = torch.arange(12).reshape(3, 4)
print(r)
r_sum = r.sum(0) # [4]
print(r_sum)

r_sum = r_sum.view(1, -1) # [1, 4]
print(r_sum)
print(r/r_sum)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]])
tensor([12, 15, 18, 21])
tensor([[12, 15, 18, 21]])
tensor([[0.0000, 0.0667, 0.1111, 0.1429],
        [0.3333, 0.3333, 0.3333, 0.3333],
        [0.6667, 0.6000, 0.5556, 0.5238]])


# `model.py`


## Student


### tokenizer

In [ ]:
from transformers import BertTokenizerFast

In [ ]:
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

In [ ]:
input_id = tokenizer("Hello, I'm a single sentence!")
print(input_id)

{'input_ids': [101, 7592, 1010, 1045, 1005, 1049, 1037, 2309, 6251, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


In [ ]:
sent = tokenizer.decode(input_id["input_ids"])
print(sent)

[CLS] hello, i'm a single sentence! [SEP]


### model

In [ ]:
import torch

In [ ]:
from model import Student

In [ ]:
# hparam = {'pretrained': 'word2vec',   # word2vec, glove, bert-base-uncased, 
#         'wv_file': './data/wv/w2v_corpus_trained.bin',     # file for word embeddings
#         'pretrained_dim': 300,
#         'num_aspect': 9,
#         'freeze_emb': True,}
hparam = {'pretrained': 'bert-base-uncased',   # word2vec, glove, bert-base-uncased, 
        'wv_file': './data/wv/w2v_corpus_trained.bin',     # file for word embeddings
        'pretrained_dim': 768,
        'num_aspect': 9,
        'freeze_emb': 1}
hparam = {'pretrained': 'bert-base-uncased',   # word2vec, glove, bert-base-uncased, 
        'wv_file': './data/wv/w2v_corpus_trained.bin',     # file for word embeddings
        'pretrained_dim': 768,
        'num_aspect': 9,
        'freeze_emb': 0}

In [ ]:
s = Student(hparam)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
print(s.fc)
for m in s.fc:
    torch.nn.init.xavier_uniform_(m.weight)
    torch.nn.init.zeros_(m.bias)

Sequential(
  (0): Linear(in_features=300, out_features=9, bias=True)
)


In [ ]:
# check if freezing embedding works well
for m, param in s.named_parameters():
    if param.requires_grad == True:
        print(m)

encoder.embeddings.word_embeddings.weight
encoder.embeddings.position_embeddings.weight
encoder.embeddings.token_type_embeddings.weight
encoder.embeddings.LayerNorm.weight
encoder.embeddings.LayerNorm.bias
encoder.encoder.layer.0.attention.self.query.weight
encoder.encoder.layer.0.attention.self.query.bias
encoder.encoder.layer.0.attention.self.key.weight
encoder.encoder.layer.0.attention.self.key.bias
encoder.encoder.layer.0.attention.self.value.weight
encoder.encoder.layer.0.attention.self.value.bias
encoder.encoder.layer.0.attention.output.dense.weight
encoder.encoder.layer.0.attention.output.dense.bias
encoder.encoder.layer.0.attention.output.LayerNorm.weight
encoder.encoder.layer.0.attention.output.LayerNorm.bias
encoder.encoder.layer.0.intermediate.dense.weight
encoder.encoder.layer.0.intermediate.dense.bias
encoder.encoder.layer.0.output.dense.weight
encoder.encoder.layer.0.output.dense.bias
encoder.encoder.layer.0.output.LayerNorm.weight
encoder.encoder.layer.0.output.LayerNorm

## Teacher

Would it be a problem to just assign 1 as appearance times? 

In [ ]:
from scipy.special import softmax
import numpy as np
x = np.zeros(5)
x[1] = 1
print('one-hot vector', x)
m = softmax(x)
print('after softmax', m)

one-hot vector [0. 1. 0. 0. 0.]
after softmax [0.14884758 0.40460968 0.14884758 0.14884758 0.14884758]


In [ ]:
# a = torch.rand(3,4)
a = torch.arange(12).view(3,4)
# b = torch.rand(4)
b = torch.arange(4)
print(a, b)
c = b * a
print(c)

tensor([[ 0,  1,  2,  3],
        [ 4,  5,  6,  7],
        [ 8,  9, 10, 11]]) tensor([0, 1, 2, 3])
tensor([[ 0,  1,  4,  9],
        [ 0,  5, 12, 21],
        [ 0,  9, 20, 33]])


In [ ]:
import torch

In [ ]:
mat = torch.tensor([[0.6, 0.0, 1, 0.0],
                    [0.0, 0.4, 0.0, 0.0],
                    [0.0, 0.0, 1.2, 0.0],
                    [0.0, 0.0, 0.0,-0.4]])
idx = torch.nonzero(mat)

In [ ]:
mat[torch.where(mat==0)]

In [ ]:
mat.max(1)

In [ ]:
torch.where(mat.max(1)[1] == 1)

# `dataset.py`

In [1]:
from dataset import *

## `TestDataset`

In [ ]:
aspect_init_file = "./data/bags_and_cases.30.txt"
data_file = "./data/bags_and_cases_test.json"

In [ ]:
test_ds = TestDataset(aspect_init_file, data_file)

loading dataset...
asp_category: 9
bag_dim: 226
length of data: 653; 	length_of_labels: 653


In [ ]:
from torch.utils.data import DataLoader

In [ ]:
test_loader = DataLoader(test_ds, batch_size=500, num_workers=8)

In [ ]:
idx, bow, labels = next(iter(test_loader))

In [ ]:
print(labels)

## `Dataset`

### data inspection

In [2]:
data_base_dir = './data/'
domain = 'bags_and_cases'
stu_params = {'pretrained': 'word2vec'}

In [3]:
ds = Dataset(data_base_dir, domain, stu_params)

INFO:root:[bags_and_cases] dataset from following files: ./data/bags_and_cases_train.json, ./data/bags_and_cases_vocab_w2v.txt, ./data/bags_and_cases_train_supplement_w2v.pkl, ./data/bags_and_cases.30.txt
INFO:root:[bags_and_cases] dataset_size: 588229
INFO:root:[bags_and_cases] number of aspects: 9
INFO:root:[bags_and_cases] number of unique seed words: 226


In [9]:
# sorted(ds.data_orig, key=len)
# ds.data_orig
# ds.data_idx
# ds.data_length

In [8]:
from nltk.tokenize import RegexpTokenizer
tokenizer = RegexpTokenizer(r'\w+')
eg_str = "you 've"
token_sent = tokenizer.tokenize(eg_str)    # "you 've"
token_sent

['you', 've']

### seed words, aspects, index

In [ ]:
aspect_init_file = "./data/boots.5.txt"
data_file = "./data/boots_train.json"

In [ ]:
ds = Dataset(aspect_init_file, data_file)

INFO:filelock:Lock 140569514659088 acquired on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


INFO:filelock:Lock 140569514659088 released on /root/.cache/huggingface/transformers/45c3f7a79a80e1cf0a489e5c62b43f173c15db47864303a55d623bb3c96f72a5.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99.lock


INFO:filelock:Lock 140572138428368 acquired on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


INFO:filelock:Lock 140572138428368 released on /root/.cache/huggingface/transformers/534479488c54aeaf9c3406f647aa2ec13648c06771ffe269edabebd4c412da1d.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4.lock


INFO:filelock:Lock 140569514555216 acquired on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock


INFO:filelock:Lock 140569514555216 released on /root/.cache/huggingface/transformers/c1d7f0a763fb63861cc08553866f1fc3e5a6f4f07621be277452d26d71303b7e.20430bd8e10ef77a7d2977accefe796051e01bc2fc4aa146bc862997a1a15e79.lock



asp_category: 9
bag_dim: 44


In [ ]:
print(ds.aspects, '\n')
print(ds.id2asp, '\n')
print(ds.asp2id, '\n')
print(ds.aspect_ids, '\n')

[['color', 'love', 'style', 'unbelievably', 'gorgeous'], ['comfortable', 'fit', 'foot', 'hurt', 'ankle'], ['rubber', 'quality', 'use', 'buckle', 'appear'], ['cute', 'look', 'looked', 'great', 'fringe'], ['leather', 'inside', 'fringe', 'material', 'heel'], ['return', 'pair', 'thought', 'year', 'boot'], ['worth', 'money', 'cheap', 'inexpensive', 'paid'], ['size', 'ordered', 'half', 'order', 'big'], ['dry', 'waterproof', 'rain', 'wet', 'water']] 

{0: 'ankle', 1: 'appear', 2: 'big', 3: 'boot', 4: 'buckle', 5: 'cheap', 6: 'color', 7: 'comfortable', 8: 'cute', 9: 'dry', 10: 'fit', 11: 'foot', 12: 'fringe', 13: 'gorgeous', 14: 'great', 15: 'half', 16: 'heel', 17: 'hurt', 18: 'inexpensive', 19: 'inside', 20: 'leather', 21: 'look', 22: 'looked', 23: 'love', 24: 'material', 25: 'money', 26: 'order', 27: 'ordered', 28: 'paid', 29: 'pair', 30: 'quality', 31: 'rain', 32: 'return', 33: 'rubber', 34: 'size', 35: 'style', 36: 'thought', 37: 'unbelievably', 38: 'use', 39: 'water', 40: 'waterproof', 41

In [ ]:
word_dict = {}
with open(aspect_init_file, 'r') as f:
    for line in f.readlines():
        l = line.strip().split(' ')
        words = l[0:]
        for w in words:
            if w in word_dict.keys():
                word_dict[w] += 1
            else:
                word_dict[w] = 1
print(word_dict)
# or use collections.Counter()

{'color': 1, 'love': 1, 'style': 1, 'unbelievably': 1, 'gorgeous': 1, 'comfortable': 1, 'fit': 1, 'foot': 1, 'hurt': 1, 'ankle': 1, 'rubber': 1, 'quality': 1, 'use': 1, 'buckle': 1, 'appear': 1, 'cute': 1, 'look': 1, 'looked': 1, 'great': 1, 'fringe': 2, 'leather': 1, 'inside': 1, 'material': 1, 'heel': 1, 'return': 1, 'pair': 1, 'thought': 1, 'year': 1, 'boot': 1, 'worth': 1, 'money': 1, 'cheap': 1, 'inexpensive': 1, 'paid': 1, 'size': 1, 'ordered': 1, 'half': 1, 'order': 1, 'big': 1, 'dry': 1, 'waterproof': 1, 'rain': 1, 'wet': 1, 'water': 1}


In [ ]:
idx2asp = ds.get_idx2asp()
n_word_per_aspect = len(ds.aspects[0])
print(idx2asp)
print(len(idx2asp))
print(idx2asp[ds.asp2id['fringe']])

[1, 2, 7, 5, 2, 6, 0, 1, 3, 8, 1, 1, 3, 0, 3, 7, 4, 1, 6, 4, 4, 3, 3, 0, 4, 6, 7, 7, 6, 5, 2, 8, 5, 2, 7, 0, 5, 0, 2, 8, 8, 8, 6, 5]
44
3


**Note**: word fringe appears twice, and it was assigned to the category in which it first appear

In [ ]:
print(len(ds))

963866


### `__getitem__` method

In [ ]:
from dataset import Dataset
import torch

In [ ]:
aspect_init_file = "./data/seedwords_30_coarse.txt"
data_file = "./data/comment_train.json"
stu_param = {'pretrained': 'word2vec',   # word2vec, glove, bert-base-uncased, 
            'wv_file': './data/wv/w2v_corpus_trained.bin',
            'pretrained_dim': 300,
            'num_aspect': 9,
            'freeze_emb': 1}

In [ ]:
ds = Dataset(aspect_init_file, data_file, stu_param)

asp_category: 6
bag_dim: 168


In [ ]:
ds[1]

(tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 tensor([   1,    6,    2, 3834,    1,    7,    1, 5353,  930,    7]),
 tensor([10]))

In [ ]:
eg = ds.data[740000]
print(eg)
bows = ds.vectorizer.transform([eg]).toarray()
print(bows)
print(np.where(bows==1))

I disagree with other reviewers in terms of size .
[[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0
  0 0 0 0 0 0 0 0]]
(array([0], dtype=int64), array([34], dtype=int64))


In [ ]:
# dataset does not support batch fetching
bow_batch, id_batch = ds[100]

# `extract_data.py`


In [ ]:
import h5py
from tqdm import tqdm
import json

In [ ]:
source = "./data/preprocessed/BOOTS_MATE.hdf5"
# source = "./data/preprocessed/BOOTS_MATE_TEST.hdf5"
output = "./data/boots_train.json"

In [ ]:
def load_h5(f, label):
    size = len(f.get(label))
    result = [f.get(label).get(str(i))[()].squeeze().tolist() for i in tqdm(range(size))]
    return result

In [ ]:
f.close()
f = h5py.File(source, 'r')

In [ ]:
print(len(f.get('original')))

19278


In [ ]:
# a = f.get('original').get('1')[()].squeeze().tolist()
a = f.get('data').get(str('12'))[()].tolist()
print(a)
print(type(a))
print(len(a))
print(a[0])
print(type(a[0]) == bytes)
isinstance(a[0], bytes)

[[983, 96, 737, 99, 1364, 5071, 976, 1033, 973, 149, 12, 8810, 381, 3821]]
<class 'list'>
1
[983, 96, 737, 99, 1364, 5071, 976, 1033, 973, 149, 12, 8810, 381, 3821]
False


False

In [ ]:
list(map(lambda x:x.decode(), f.get('original').get('0')[()].squeeze().tolist()))

['that I love ,',
 'but these are a disappointment .',
 'I do not like them at all ,',
 'returning them .',
 'They are not comfortable .',
 'they gave me .',
 'how i will wrap these .',
 'I requested .',
 'They said',
 'This is the day',
 'that was the boots',
 "do n't have !",
 'Thanks',
 'I bought',
 'wide',
 'that ok am',
 'enjoying them',
 'and they were too small .',
 'I thought',
 'I like them .',
 'but i guess',
 'which is good !',
 'But i love them',
 'they were too big .',
 'to try them out .',
 'Do remember',
 'after breaking them in , so',
 '1 .',
 '2 .',
 '3 .',
 '4 .',
 'she wanted .',
 'It took a while',
 'to get them ;',
 'I guess',
 'to replace them .',
 'I buy',
 'and have to say',
 'So , I bought',
 'I know how',
 "but did n't .",
 'it was the boots',
 'I wore them',
 'To be fair ,',
 'what I need to do',
 'Ridiculous !',
 'and try them on .',
 'so it swallowed me up :',
 'and the boots',
 'It was absurd -']

In [ ]:
data = {'original': load_h5(f, 'original')}
data['original'] = [[o] if isinstance(o, str) else o for o in data['original']]

100%|██████████| 19278/19278 [00:06<00:00, 2934.68it/s]


In [ ]:
print(type(data['original']))
for i in range(10):
    print(data['original'][i])

<class 'list'>
[b'that I love ,', b'but these are a disappointment .', b'I do not like them at all ,', b'returning them .', b'They are not comfortable .', b'they gave me .', b'how i will wrap these .', b'I requested .', b'They said', b'This is the day', b'that was the boots', b"do n't have !", b'Thanks', b'I bought', b'wide', b'that ok am', b'enjoying them', b'and they were too small .', b'I thought', b'I like them .', b'but i guess', b'which is good !', b'But i love them', b'they were too big .', b'to try them out .', b'Do remember', b'after breaking them in , so', b'1 .', b'2 .', b'3 .', b'4 .', b'she wanted .', b'It took a while', b'to get them ;', b'I guess', b'to replace them .', b'I buy', b'and have to say', b'So , I bought', b'I know how', b"but did n't .", b'it was the boots', b'I wore them', b'To be fair ,', b'what I need to do', b'Ridiculous !', b'and try them on .', b'so it swallowed me up :', b'and the boots', b'It was absurd -']
[b'The pair', b'No matter ,', b'And , they .

In [ ]:
data['label'] = load_h5(f, 'labels')
data['label'] = [[o] if isinstance(o[0], int) else o for o in data['label']]

TypeError: object of type 'NoneType' has no len()

In [ ]:
with open(output, 'w') as f:
    json.dump(data, f)

TypeError: Object of type bytes is not JSON serializable

# MISC

In [1]:
import json

In [2]:
%cd /content/drive/MyDrive/LeverageJustAFewKeywords/

/content/drive/MyDrive/LeverageJustAFewKeywords


In [4]:
train_file = './data/bags_and_cases_train.json'
with open(train_file, 'r') as f:
    original = json.load(f)

In [6]:
total_length = 0
for batch in original['original']:
    total_length += len(batch)
total_length

588229